In [30]:
import spikeinterface.full as si
import numpy as np
import pandas as pd
from pathlib import Path
from spikeinterface.generation.drift_tools import DriftingTemplates, interpolate_templates
import pickle

In [17]:
### RE recordings (destriped)


pids = [
        "1a276285-8b0e-4cc9-9f0a-a3a002978724",
        "1e104bf4-7a24-4624-a5b2-c2c8289c0de7",
        "5d570bf6-a4c6-4bf1-a14b-2c878c84ef0e",
        "5f7766ce-8e2e-410c-9195-6bf089fea4fd",
        "6638cfb3-3831-4fc2-9327-194b76cf22e1",
        "749cb2b7-e57e-4453-a794-f6230e4d0226",
        "d7ec0892-0a6c-4f4f-9d8f-72083692af5c",
        "da8dfec1-d265-44e8-84ce-6ae9c109b8bd",
        "dab512bd-a02d-4c1f-8dbc-9155a163efc0",
        "dc7e9403-19f7-409f-9240-05ee57cb7aea",
        "decc8d40-cf74-4263-ae9d-a0cc68b47e86",
        "e8f9fba4-d151-4b00-bee7-447f0f3e752c",
        "eebcaf65-7fa4-4118-869d-a084e84530e2",
        "fe380793-8035-414e-b000-09bfe5ece92a"
       ]

def rec_from_pid(pid):
    recordings_path = Path("/expanse/lustre/projects/col168/cwindolf/benchmark_recordings/")
    ppath = recordings_path.joinpath(pid)
    return si.read_binary_folder(ppath)

In [23]:
### RE templates


temp_pids = ["02cc03e4-8015-4050-bb42-6c832091febb",
"0aafb6f1-6c10-4886-8f03-543988e02d9e",
"0b8ea3ec-e75b-41a1-9442-64f5fbc11a5a",
"31f3e083-a324-4b88-b0a4-7788ec37b191",
"41a3b948-13f4-4be7-90b9-150705d39005",
"63517fd4-ece1-49eb-9259-371dc30b1dd6",
"84bb830f-b9ff-4e6b-9296-f458fb41d160",
"9117969a-3f0d-478b-ad75-98263e3bfacf",
"c07d13ed-e387-4457-8e33-1d16aed3fd92",
"e31b4e39-e350-47a9-aca4-72496d99ff2a",
    ]

def templates_from_pid(pid, return_all=False):
    templates_path = Path("/expanse/lustre/projects/col168/cwindolf/rs_templates/")
    tpath = templates_path.joinpath(f"temps{pid}", "template_data.npz")
    z = np.load(tpath)
    if return_all:
        return z["templates"], z["unit_ids"], z["spike_counts"]
    return z["templates"]

In [31]:
### Motion estimates


motion_pids = ["1e176f17-d00f-49bb-87ff-26d237b525f1",
        "8ca1a850-26ef-42be-8b28-c2e2d12f06d6",
        "c4f6665f-8be5-476b-a6e8-d81eeae9279d",
        "dab512bd-a02d-4c1f-8dbc-9155a163efc0",
        "e31b4e39-e350-47a9-aca4-72496d99ff2a",
]

def motion_est_from_pid(pid):
    motion_path = Path("/expanse/lustre/projects/col168/cwindolf/rs_hybrid/original_thresholding")
    ppath = motion_path.joinpath(pid, "motion_est.pkl")
    with open(ppath, 'rb') as f:
        x = pickle.load(f)
    return x


In [38]:
### Start analysis

# pids we have preprocessed data for and motion est for 
pids_use = list(set(pids).intersection(set(motion_pids)))

In [51]:
### Create drifty templates
pid = pids_use[0]

templates = templates_from_pid(temp_pids[3], return_all=True)

me = motion_est_from_pid(pid)

In [52]:
templates

(array([[[-9.37393308e-02, -2.18434155e-01, -3.16047221e-02, ...,
          -1.61222219e-02,  8.45345203e-03,  8.94172210e-03],
         [-7.06680417e-02, -2.23950863e-01, -1.49569381e-02, ...,
          -1.31285014e-02,  1.09356856e-02,  7.79568392e-04],
         [-5.37430570e-02, -1.89368010e-01,  1.04920734e-02, ...,
          -1.13412691e-02,  6.52968884e-03,  1.63020268e-05],
         ...,
         [-5.92356585e-02,  2.33754702e-02, -7.81680867e-02, ...,
           1.43469300e-03, -1.27405999e-03,  8.50094948e-03],
         [-4.21281457e-02,  4.64196801e-02, -4.43007946e-02, ...,
           7.36153102e-04, -3.96986771e-03,  4.39357199e-03],
         [-1.68156549e-02,  4.70886081e-02, -2.09115855e-02, ...,
          -2.88140523e-04, -3.58813070e-03,  3.72084836e-03]],
 
        [[-1.80872783e-01, -1.84478965e-02, -2.34175935e-01, ...,
           1.71230659e-02,  2.29219976e-03, -6.61149295e-03],
         [-1.75810397e-01, -3.39600537e-03, -2.42719799e-01, ...,
           2.46770196